In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix, roc_curve
from sklearn import metrics
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
Olaparib_1017 = pd.read_csv('./drug_respond/data/smmart_protein_rna_tissue10_ready/Olaparib_1017.tsv.gz', sep='\t', index_col=0)
Olaparib_1017

,depmap_ID,DRUG_ID,DRUG_NAME,Primary Disease,Gender,AUC,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),...,NUDT3 (ENSG00000272325),C6ORF165 (ENSG00000272514),MUSTN1 (ENSG00000272573),DOC2B (ENSG00000272636),RP11-309L24.9 (ENSG00000272899),C2ORF15 (ENSG00000273045),GRIN2B (ENSG00000273079),SNURF (ENSG00000273173),RP11-1263C18.1 (ENSG00000273238),ZBTB8B (ENSG00000273274)
5,ACH-000087,1017,Olaparib,Bone Cancer,Male,0.846324,4.955127,0.070389,5.873567,2.853996,...,4.694880,0.137504,0.545968,3.203201,0.978196,0.669027,0.000000,7.092229,0.669027,0.042644
6,ACH-000327,1017,Olaparib,Lung Cancer,Female,0.970014,4.041769,0.000000,5.813525,1.918386,...,3.730096,0.028569,0.070389,0.000000,1.035624,2.310340,0.000000,0.641546,0.000000,0.000000
7,ACH-000905,1017,Olaparib,Bladder Cancer,Male,0.915457,5.881175,0.000000,6.643135,2.039138,...,2.664483,0.084064,0.226509,0.000000,0.422233,2.003602,0.000000,5.660210,0.000000,0.432959
8,ACH-000242,1017,Olaparib,Bladder Cancer,Male,0.987345,7.496574,0.000000,6.458940,2.364572,...,3.658783,0.739848,0.201634,0.000000,0.378512,3.557042,0.000000,6.843607,0.000000,0.948601
9,ACH-000384,1017,Olaparib,Bladder Cancer,Female,0.952230,5.755155,0.000000,6.928844,2.895303,...,3.324811,2.042644,1.531069,0.000000,0.622930,3.244887,0.000000,5.405312,0.000000,1.077243
13,ACH-000323,1017,Olaparib,Brain Cancer,Male,0.922807,4.590362,0.000000,6.647890,1.899176,...,3.187451,0.111031,0.526069,0.000000,0.298658,0.176323,0.000000,6.463524,0.650765,0.014355
14,ACH-000137,1017,Olaparib,Brain Cancer,Female,0.901520,5.108106,0.000000,6.888500,2.207893,...,3.861955,0.137504,0.879706,0.000000,0.333424,1.490570,0.056584,7.145372,0.000000,0.757023
15,ACH-000738,1017,Olaparib,Brain Cancer,Male,0.901069,4.078951,0.000000,6.691953,2.121015,...,3.056584,0.084064,0.855990,0.000000,0.536053,0.505891,0.014355,6.101818,0.000000,0.704872
16,ACH-000571,1017,Olaparib,Brain Cancer,Male,0.960333,3.408712,0.000000,6.677931,2.269033,...,3.247928,0.124328,0.000000,0.000000,0.731183,0.495695,0.000000,0.669027,0.000000,0.028569
17,ACH-000040,1017,Olaparib,Brain Cancer,Male,0.983662,3.828835,0.000000,6.436295,2.073820,...,3.782409,0.526069,0.516015,0.000000,1.021480,0.687061,0.014355,6.302685,0.000000,0.000000


In [4]:
X, y = Olaparib_1017.iloc[:, 6:].values, Olaparib_1017.iloc[:, 5].values
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Cellline No.'
    ),
    yaxis=dict(
        title='Drug response curve AUC'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [5]:
predicted = pd.read_csv('./Galaxy10-[Cross_val_predict_on_collection_9_and_Olaparib_1017__Pipeline Builder on data 18 -- BinarizeTargetRegressor (XGBRegressor)].tabular', sep='\t')
predicted

,Predicted
0,0.068004
1,0.035193
2,0.041462
3,0.027545
4,0.027277
5,0.053965
6,0.029784
7,0.018503
8,0.039451
9,0.043558


In [6]:
combined_df = Olaparib_1017.iloc[:, :6]
combined_df['Predicted'] = 1 - predicted['Predicted'].values
combined_df['1-prediction'] = predicted['Predicted'].values
combined_df = combined_df.reset_index(drop=True)
combined_df

,depmap_ID,DRUG_ID,DRUG_NAME,Primary Disease,Gender,AUC,Predicted,1-prediction
0,ACH-000087,1017,Olaparib,Bone Cancer,Male,0.846324,0.931996,0.068004
1,ACH-000327,1017,Olaparib,Lung Cancer,Female,0.970014,0.964807,0.035193
2,ACH-000905,1017,Olaparib,Bladder Cancer,Male,0.915457,0.958538,0.041462
3,ACH-000242,1017,Olaparib,Bladder Cancer,Male,0.987345,0.972455,0.027545
4,ACH-000384,1017,Olaparib,Bladder Cancer,Female,0.952230,0.972723,0.027277
5,ACH-000323,1017,Olaparib,Brain Cancer,Male,0.922807,0.946035,0.053965
6,ACH-000137,1017,Olaparib,Brain Cancer,Female,0.901520,0.970216,0.029784
7,ACH-000738,1017,Olaparib,Brain Cancer,Male,0.901069,0.981497,0.018503
8,ACH-000571,1017,Olaparib,Brain Cancer,Male,0.960333,0.960549,0.039451
9,ACH-000040,1017,Olaparib,Brain Cancer,Male,0.983662,0.956442,0.043558


In [7]:
discretize = combined_df['AUC'].mean() - combined_df['AUC'].std()
y_true = combined_df['AUC'].values < discretize


## roc_curve by `1-predicted`

In [8]:
fpr, tpr, thresholds = roc_curve(y_true, combined_df['1-prediction'].values)
fpr, tpr, thresholds

(array([0.        , 0.        , 0.0042735 , 0.0042735 , 0.01282051,
        0.01282051, 0.02350427, 0.02350427, 0.02777778, 0.02777778,
        0.02991453, 0.02991453, 0.03632479, 0.03632479, 0.04059829,
        0.04059829, 0.04273504, 0.04273504, 0.05555556, 0.05555556,
        0.05982906, 0.05982906, 0.08547009, 0.08547009, 0.13034188,
        0.13034188, 0.13675214, 0.13675214, 0.1517094 , 0.1517094 ,
        0.16666667, 0.16666667, 0.17735043, 0.17735043, 0.19871795,
        0.19871795, 0.22863248, 0.22863248, 0.25213675, 0.25213675,
        0.31623932, 0.31623932, 0.33760684, 0.33760684, 0.37820513,
        0.37820513, 0.42307692, 0.42307692, 0.45940171, 0.45940171,
        0.61538462, 0.61538462, 0.63034188, 0.63034188, 0.66025641,
        0.66025641, 0.69230769, 0.69230769, 0.6965812 , 0.6965812 ,
        0.73931624, 0.73931624, 0.75641026, 0.75641026, 0.77564103,
        0.77564103, 0.79487179, 0.79487179, 0.8482906 , 0.8482906 ,
        0.88461538, 0.88461538, 0.88888889, 0.88

## new roc_curve by ranking predicted value directly

In [9]:
combined_df_sort = combined_df.sort_values(['Predicted'])
combined_df_sort

,depmap_ID,DRUG_ID,DRUG_NAME,Primary Disease,Gender,AUC,Predicted,1-prediction
188,ACH-000346,1017,Olaparib,Leukemia,Male,0.869765,0.656655,0.343345
273,ACH-000719,1017,Olaparib,Ovarian Cancer,Female,0.983181,0.683982,0.316018
108,ACH-000148,1017,Olaparib,Breast Cancer,Female,0.977249,0.692931,0.307069
343,ACH-000008,1017,Olaparib,Skin Cancer,Male,0.861701,0.697281,0.302719
294,ACH-000236,1017,Olaparib,Colon/Colorectal Cancer,Female,0.986585,0.700048,0.299952
482,ACH-000745,1017,Olaparib,Myeloma,Male,0.972036,0.724239,0.275761
483,ACH-000129,1017,Olaparib,Lung Cancer,Female,0.990768,0.794642,0.205358
70,ACH-000703,1017,Olaparib,Lung Cancer,Male,0.969030,0.814571,0.185429
328,ACH-000432,1017,Olaparib,Leukemia,Male,0.746874,0.823344,0.176656
436,ACH-000178,1017,Olaparib,Pancreatic Cancer,Male,0.938656,0.848600,0.151400


In [10]:
total_positive = np.sum(y_true)
total_negative = combined_df_sort.shape[0] - total_positive
new_thresholds = np.r_[0, combined_df_sort['Predicted'].values, 1]
new_tprs = []
new_fprs = []
for thres in new_thresholds:
    #predicted_positive
    p_df = combined_df_sort[combined_df_sort['Predicted'] < thres]
    # true positive count
    tp_count = np.sum(p_df['AUC'] < discretize)
    # false positive count
    fp_count = p_df.shape[0] - tp_count
    # get rates
    new_tprs.append(tp_count/total_positive)
    new_fprs.append(fp_count/total_negative)

In [11]:
new_tprs, new_fprs

([0.0,
  0.0,
  0.02127659574468085,
  0.02127659574468085,
  0.02127659574468085,
  0.0425531914893617,
  0.0425531914893617,
  0.0425531914893617,
  0.0425531914893617,
  0.0425531914893617,
  0.06382978723404255,
  0.06382978723404255,
  0.06382978723404255,
  0.06382978723404255,
  0.06382978723404255,
  0.06382978723404255,
  0.0851063829787234,
  0.0851063829787234,
  0.0851063829787234,
  0.10638297872340426,
  0.1276595744680851,
  0.1276595744680851,
  0.14893617021276595,
  0.14893617021276595,
  0.14893617021276595,
  0.14893617021276595,
  0.1702127659574468,
  0.1702127659574468,
  0.1702127659574468,
  0.19148936170212766,
  0.19148936170212766,
  0.2127659574468085,
  0.2127659574468085,
  0.2127659574468085,
  0.2127659574468085,
  0.2127659574468085,
  0.2127659574468085,
  0.2127659574468085,
  0.23404255319148937,
  0.23404255319148937,
  0.23404255319148937,
  0.2553191489361702,
  0.2553191489361702,
  0.2553191489361702,
  0.2553191489361702,
  0.2553191489361702,

In [12]:
data2 = go.Scatter(
        x = fpr,
        y = tpr,
        mode = "lines",
        name = "roc_curve",
)

data3 = go.Scatter(
        x = [0, 1],
        y = [0, 1],
        mode = 'lines',
        name = 'x=y' 
)

data4 = go.Scatter(
        x = new_fprs,
        y = new_tprs,
        mode = "lines",
        name = "roc_curve_new",
)

layout = dict(
    xaxis=dict(
        title='False Positive Rate'
    ),
    yaxis=dict(
        title='True Positive Rate'
    ),
    title="ROC Curve"
    
)
fig = go.Figure(data=[data2, data3,data4], layout=layout)
iplot(fig)